In [1]:
import pymysql, os, copy, json, time
import pandas as pd
import numpy as np
from scipy.stats import mode

In [2]:
def _call_db_info(db_name):
    return pymysql.connect(
        host = 'trend.cb7jqghocrtb.ap-northeast-2.rds.amazonaws.com',
        port= 3306,
        user = 'root',
        password='ensol2020!',
        db = db_name,
        charset = 'utf8')

In [3]:
conn = _call_db_info('gscm_e61d')
curs = conn.cursor()
curs.execute("select * from dt_table;") 
dt_table = pd.DataFrame(curs.fetchall())
conn.commit()
conn.close()
dt_table.columns = ['No', 'WW']    
#dt_table = dt_table['WW']
dt_table.dtypes

No    int64
WW    int64
dtype: object

In [4]:
start = 202001

## Inventory Result

In [134]:
insert_inven_sql = "insert into inven_rslt values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"#.format(*c)
tmp = pd.read_csv('./data1/Inven_3xxC.csv')
ids = tmp.index[tmp['Inven Week'] == start].tolist()[0]

conn = _call_db_info('gscm_e61d')
curs = conn.cursor()
for i in range(ids, tmp.shape[0]):
    tmp_s = dt_table.index[dt_table['WW']==tmp['Inven Week'].loc[i]].tolist()[0]    
    tmp_e = tmp_s - 8    
    ww_candidate = dt_table.loc[tmp_e:tmp_s].reset_index(drop=True)  
    #print(ww_candidate)
    tmp_join = pd.merge(ww_candidate, tmp, left_on='WW', right_on='Inven Week')
    tmp_join = tmp_join.sort_values('WW', ascending=False)     
    #print(tmp_join)
    inven_list = list(tmp_join[tmp_join['WW']==tmp['Inven Week'].loc[i]][['Inven Week', 'Date']].values.reshape(-1))
    inven_list.extend(tmp_join['재고수량'].values.tolist())
    print(i, inven_list)
    #break;
    curs.execute(insert_inven_sql, inven_list) 
conn.commit()
conn.close()    

52 [202001, '2020-01-05', 629709, 704804, 660010, 548225, 453382, 402367, 514039, 503889, 583257]
53 [202002, '2020-01-12', 649121, 629709, 704804, 660010, 548225, 453382, 402367, 514039, 503889]
54 [202003, '2020-01-19', 595718, 649121, 629709, 704804, 660010, 548225, 453382, 402367, 514039]
55 [202004, '2020-01-26', 561899, 595718, 649121, 629709, 704804, 660010, 548225, 453382, 402367]
56 [202005, '2020-02-02', 592359, 561899, 595718, 649121, 629709, 704804, 660010, 548225, 453382]
57 [202006, '2020-02-09', 626276, 592359, 561899, 595718, 649121, 629709, 704804, 660010, 548225]
58 [202007, '2020-02-16', 537030, 626276, 592359, 561899, 595718, 649121, 629709, 704804, 660010]
59 [202008, '2020-02-23', 433067, 537030, 626276, 592359, 561899, 595718, 649121, 629709, 704804]
60 [202009, '2020-03-01', 232891, 433067, 537030, 626276, 592359, 561899, 595718, 649121, 629709]
61 [202010, '2020-03-08', 390616, 232891, 433067, 537030, 626276, 592359, 561899, 595718, 649121]
62 [202011, '2020-03

## Product Result

In [18]:
insert_prod_sql = "insert into prod_rslt values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
tmp = pd.read_csv('./data1/DynamicResult.csv')
tmp = tmp.drop_duplicates().reset_index(drop=True)

ids = tmp.index[tmp['Plan Week'] == start].tolist()[0]

conn = _call_db_info('gscm_e61d')
curs = conn.cursor()
for i in range(ids, tmp.shape[0]):
    tmp_s = dt_table.index[dt_table['WW']==tmp['Plan Week'].loc[i]].tolist()[0]    
    tmp_e = tmp_s - 12
    ww_candidate = dt_table.loc[tmp_e:tmp_s].reset_index(drop=True)  
    tmp_join = pd.merge(ww_candidate, tmp, how='left',  left_on='WW', right_on='Plan Week')    
    
    tmp_join = tmp_join.sort_values('WW', ascending=False)
    tmp_join = tmp_join.fillna(0)  
    tmp_join = tmp_join.astype({'Result Qty' : 'int'})
    #print(tmp_join)
    prod_list = list(tmp_join[tmp_join['WW']==tmp['Plan Week'].loc[i]][['WW', 'Basic Date']].values.reshape(-1))
    #print(prod_list)
    prod_list.extend(tmp_join['Result Qty'])    
    print(i, prod_list)
    #break;
    curs.execute(insert_prod_sql, prod_list) 
conn.commit()
conn.close()      

38 [202001, '2020-01-05', 88080, 116400, 209658, 179483, 76128, 0, 132619, 55318, 99337, 182218, 176397, 144834, 0]
39 [202002, '2020-01-12', 143332, 88080, 116400, 209658, 179483, 76128, 0, 132619, 55318, 99337, 182218, 176397, 144834]
40 [202003, '2020-01-19', 126497, 143332, 88080, 116400, 209658, 179483, 76128, 0, 132619, 55318, 99337, 182218, 176397]
41 [202004, '2020-01-26', 46482, 126497, 143332, 88080, 116400, 209658, 179483, 76128, 0, 132619, 55318, 99337, 182218]
42 [202005, '2020-02-02', 192274, 46482, 126497, 143332, 88080, 116400, 209658, 179483, 76128, 0, 132619, 55318, 99337]
43 [202006, '2020-02-09', 101129, 192274, 46482, 126497, 143332, 88080, 116400, 209658, 179483, 76128, 0, 132619, 55318]
44 [202008, '2020-02-23', 0, 0, 101129, 192274, 46482, 126497, 143332, 88080, 116400, 209658, 179483, 76128, 0]
45 [202010, '2020-03-08', 193220, 0, 0, 0, 101129, 192274, 46482, 126497, 143332, 88080, 116400, 209658, 179483]
46 [202011, '2020-03-15', 185859, 193220, 0, 0, 0, 10112

## Product Plan

In [23]:
insert_prod_sql = "insert into prod_plan values (%s, %s, %s)"
tmp = pd.read_csv('./data1/02.Detail_생산계획 준수율.csv')
tmp = tmp.drop_duplicates().reset_index(drop=True)
current = max(tmp['WW'])

tmp_s = dt_table.index[dt_table['WW'] == start].tolist()[0]
tmp_e = dt_table.index[dt_table['WW'] == current].tolist()[0]

conn = _call_db_info('gscm_e61d')
curs = conn.cursor()
for i in range(tmp_s, (tmp_e+1)):    
    tmp_ = tmp[tmp['WW'] == dt_table['WW'].loc[i]].reset_index(drop=True)
    tmp_ = tmp_.sort_values('tWW')
    #print(tmp_)
    tmp_plan = list(map(lambda x : [str(tmp_['tWW'].values[x]), str(tmp_['QTY'].values[x])], range(0, tmp_.shape[0])))
    tmp_plan_join = str('/'.join(list(map(lambda x : ':'.join(tmp_plan[x]), range(0, tmp_.shape[0])))))
    prod_plan = list(tmp_[['WW','Date']].loc[0].tolist())
    prod_plan.append(tmp_plan_join)
    print(i, prod_plan)
    #break;
    curs.execute(insert_prod_sql, prod_plan) 
conn.commit()
conn.close() 


52 [202001, '2019-12-30', '202001:126573/202002:202500/202003:199500/202004:202500/202005:203375/202009:275875/202010:282625/202011:282625/202012:282625/202013:282625/202014:285000/202015:148000/202016:149530/202017:145530/202018:145530/202019:187110/202020:405810/202021:405810/202022:405810/202023:429300/202024:429300/202025:429300/202026:429300/202027:429300/202028:309700/202029:309700/202030:309700/202031:309700/202032:311600/202033:311600/202034:311600/202035:442800/202036:442800/202037:442800/202038:442800/202039:442800/202040:442800/202041:449550/202042:449550/202043:449550/202044:449550/202045:449550/202046:449550/202047:449550/202048:449550/202049:449550/202050:61179/202051:62088/202052:270886/202053:3281086']
53 [202002, '2020-01-06', '202002:135861/202003:200500/202004:199100/202005:171000/202009:70000/202010:282625/202011:282625/202012:282625/202013:282625/202014:282625/202015:39400/202016:132975/202017:136975/202018:132975/202019:330075/202020:330075/202021:330075/202022:33

## Product Prev

In [26]:
insert_prod_sql = "insert into prod_prev values (%s, %s, %s)"
tmp = pd.read_csv('./data1/02.Detail_생산계획 준수율.csv')
tmp = tmp.drop_duplicates().reset_index(drop=True)
current = max(tmp['WW'])

tmp_s = dt_table.index[dt_table['WW'] == start].tolist()[0]
tmp_e = dt_table.index[dt_table['WW'] == current].tolist()[0]

conn = _call_db_info('gscm_e61d')
curs = conn.cursor()
for i in range(tmp_s, (tmp_e+1)):#(tmp_s+1)):    #
    tmp_ = tmp[tmp['tWW'] == dt_table['WW'].loc[i]].reset_index(drop=True)
    tmp_ = tmp_.sort_values('WW')[::-1].reset_index(drop=True)
    tmp_prev = list(map(lambda x : [str(tmp_['WW'].values[x]), str(tmp_['QTY'].values[x])], range(1, tmp_.shape[0])))
    tmp_prev_join = str('/'.join(list(map(lambda x : ':'.join(tmp_prev[x]), range(0, len(tmp_prev))))))
    prod_prev = list(tmp_[['tWW','Date']].loc[0].tolist())
    prod_prev.append(tmp_prev_join)
    print(i, prod_prev)
    curs.execute(insert_prod_sql, prod_prev) 
conn.commit()
conn.close() 

52 [202001, '2019-12-30', '201952:151000/201951:139000/201950:206000/201949:206000/201948:208000/201947:214500/201946:215000/201945:217000/201944:215000/201943:215000/201942:214500/201941:214500/201940:209960/201939:197615/201938:215500/201937:227000/201936:227000/201935:227000/201934:227000/201933:227000/201931:227000/201930:227000/201929:227000/201928:227000/201927:227000/201926:227000/201925:227000/201924:227000/201923:242900/201921:242900/201920:242900/201919:485800/201918:485800/201917:485800/201916:485800/201915:485800/201914:485800/201913:492800/201912:492800/201911:423400/201910:396850/201909:246400/201908:281800/201907:281800/201906:159229/201905:281400/201904:281400/201903:281800/201902:281400/201901:1150708']
53 [202002, '2020-01-06', '202001:202500/201952:202500/201951:202500/201950:195000/201949:195000/201948:195000/201947:207200/201946:207225/201945:199550/201944:199550/201943:199550/201942:199550/201941:199550/201940:199550/201939:177600/201938:199550/201937:214500/20193

## Sale Plan

In [51]:
insert_sale_sql = "insert into sale_plan values (%s, %s, %s)"
f_list = os.listdir('./data1/')
f_list.sort()
sale_file = [i for i, v in enumerate(f_list) if 'Fcst' in v.split(' ')]
start = 202001
current = 202130

tmp_s = dt_table.index[dt_table['WW']==start].tolist()[0]#-12
tmp_e = dt_table.index[dt_table['WW']==current].tolist()[0]

conn = _call_db_info('gscm_e61d')
curs = conn.cursor()
for i in range(tmp_s, (tmp_e)):
    tmp_ww = dt_table['WW'].loc[i]
    cnt = 0
    for j in range(0, len(sale_file)):#tmp_s, tmp_e):    
        tmp = pd.read_csv('./data1/'+f_list[sale_file[j]])
        if tmp_ww in tmp['WW'].values and str(tmp_ww) in tmp.columns:
            #if tmp_ww in tmp['WW'] and tmp_ww in tmp.columns
            row_id = tmp.index[tmp['WW'] == tmp_ww][0]
            col_id = list(tmp.columns).index(str(tmp_ww))        
            if cnt == 0 :
                tmp_ww_plan = tmp.iloc[row_id, col_id:]
            else:
                tmp_ww_plan = pd.concat([tmp_ww_plan, tmp.iloc[row_id, col_id:]])
            
            cnt+=1
            tmp_date = tmp[tmp['WW'] == tmp_ww]['Date'].values[0]
    tmp_ww_plan = tmp_ww_plan.reset_index(drop=False).drop_duplicates().sort_values('index')    
    tmp_ww_group = tmp_ww_plan.groupby('index')
    tmp_ww_rslt = pd.DataFrame(tmp_ww_group.max()).reset_index(drop=False)    
    
    tmp_ww_rslt.columns=['ww', 'qty']
    tmp_plan = list(map(lambda x : [str(tmp_ww_rslt['ww'].values[x]), str(tmp_ww_rslt['qty'].values[x])], range(0, tmp_ww_rslt.shape[0])))
    tmp_plan_join = str('/'.join(list(map(lambda x : ':'.join(tmp_plan[x]), range(0, tmp_ww_rslt.shape[0])))))
    
    sale_plan = list([tmp_ww,tmp_date])#list(tmp[['WW', 'Date']].loc[row_id].values.reshape(-1))
    sale_plan.append(tmp_plan_join)
    print(i, sale_plan)
    curs.execute(insert_sale_sql, sale_plan) 
conn.commit()
conn.close()

134
52 [202001, '2019-12-31', '202001:60480/202002:105840/202003:117840/202004:120960/202005:120960/202006:120960/202007:120960/202008:120960/202009:120960/202010:120960/202011:120960/202012:191273/202013:192673/202014:0']
53 [202002, '2020-01-07', '202002:136080/202003:114440/202004:120960/202005:120960/202006:120960/202007:120960/202008:120960/202009:120960/202010:120960/202011:120960/202012:151200/202013:152600/202014:0']
54 [202003, '2020-01-14', '202003:129560/202004:105840/202005:120960/202006:120960/202007:141960/202008:120960/202009:120960/202010:120960/202011:120960/202012:151200/202013:152600/202014:0']
55 [202004, '2020-01-21', '202004:120960/202005:120960/202006:126085/202007:141960/202008:136080/202009:136080/202010:120960/202011:120960/202012:151200/202013:152600/202014:0']
56 [202005, '2020-01-28', '202005:120960/202006:141205/202007:141960/202008:136080/202009:136080/202010:136080/202011:105840/202012:151200/202013:152600/202014:0']
57 [202006, '2020-02-04', '202006:141

## Sale Result

In [55]:
insert_sale_sql = "insert into sale_rslt values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
f_list = os.listdir('./data1/')
f_list.sort()
sale_file = [i for i, v in enumerate(f_list) if 'Fcst' in v.split(' ')]
start = 202001
current = 202130

tmp_s = dt_table.index[dt_table['WW']==start].tolist()[0]
tmp_e = dt_table.index[dt_table['WW']==current].tolist()[0]
tmp_rslt = []
for i in range((tmp_s-12), (tmp_e)):
    tmp_ww = dt_table['WW'].loc[i]    
    tmp_ww_rslt = []
    for j in range(0, len(sale_file)):
        tmp = pd.read_csv('./data1/'+f_list[sale_file[j]])
        if str(tmp_ww) in tmp.columns:
            col_id = list(tmp.columns).index(str(tmp_ww))     
            row_id = tmp.index[tmp['WW'] > tmp_ww].values.tolist()  
            tmp_ww_rslt.extend(tmp.iloc[row_id, col_id])
            if any(tmp_ww == tmp['WW']):
                tmp_date = tmp[tmp['WW'] == tmp_ww]['Date'].values[0]
    if np.sum(tmp_ww_rslt) > 0 :
        tmp_mode = mode(np.array(tmp_ww_rslt)[np.array(tmp_ww_rslt) > 0]).mode[0]            
    else:
        tmp_mode = 0
    tmp_rslt.append([tmp_ww, tmp_date, tmp_mode])
tmp_rslt = pd.DataFrame(tmp_rslt)
print(tmp_rslt)


conn = _call_db_info('gscm_e61d')
curs = conn.cursor()
for i in range(tmp_s, (tmp_e)):
    tmp_ww = dt_table['WW'].loc[i]
    tmp_ind = tmp_rslt[tmp_rslt[0] == tmp_ww].index[0]
    tmp_sale_rslt = tmp_rslt.iloc[(tmp_ind-12):tmp_ind, 2][::-1].values.tolist()
    sale_rslt = tmp_rslt.iloc[tmp_ind, :].values.tolist()
    sale_rslt.extend(tmp_sale_rslt)    
    curs.execute(insert_sale_sql, sale_rslt) 
conn.commit()
conn.close()


         0           1       2
0   201941  2019-10-08   54432
1   201942  2019-10-15   82404
2   201943  2019-10-22   64638
3   201944  2019-10-29   12474
4   201945  2019-11-05   87696
..     ...         ...     ...
89  202125  2021-06-22     378
90  202126  2021-06-29  211656
91  202127  2021-07-06  181239
92  202128  2021-07-13   30240
93  202129  2021-07-20       0

[94 rows x 3 columns]


## Sale Prev

In [58]:
insert_sale_sql = "insert into sale_prev values (%s, %s, %s)"
f_list = os.listdir('./data1/')
f_list.sort()
sale_file = [i for i, v in enumerate(f_list) if 'Fcst' in v.split(' ')]
start = 202001
current = 202130
tmp_s = dt_table.index[dt_table['WW']==start].tolist()[0]#-12
tmp_e = dt_table.index[dt_table['WW']==current].tolist()[0]
print(tmp_s, tmp_e)

conn = _call_db_info('gscm_e61d')
curs = conn.cursor()

for i in range(tmp_s, (tmp_e)):
    tmp_ww = dt_table['WW'].loc[i]    
    cnt = 0
    for j in range(0, len(sale_file)):#tmp_s, tmp_e):    
        tmp = pd.read_csv('./data1/'+f_list[sale_file[j]])    
        if tmp_ww in tmp['WW'].values and str(tmp_ww) in tmp.columns:        
            row_id = tmp.index[tmp['WW'].values < tmp_ww].tolist()
            col_id = list(tmp.columns).index(str(tmp_ww))  
            tmp_ = tmp[['WW','Date']].copy()#.iloc[row_id]#,[0,1,col_id]]
            tmp_['tmp_prev'] = tmp.iloc[:,col_id]
            if any(tmp_ww == tmp['WW']):
                tmp_date = tmp[tmp['WW'] == tmp_ww]['Date'].values[0]
            if cnt == 0 :
                tmp_prev_plan = tmp_.iloc[row_id, :]
            else:
                tmp_prev_plan = pd.concat([tmp_prev_plan, tmp_.iloc[row_id, :]])
            cnt+=1
    tmp_prev_plan_group = tmp_prev_plan.groupby(['WW', 'Date'])
    #tmp_prev_plan_ = tmp_prev_plan.drop_duplicates().sort_values('WW', ascending=False).reset_index(drop=True)
    tmp_prev_plan_ = pd.DataFrame(tmp_prev_plan_group.max()).sort_values('WW', ascending=False).reset_index(drop=False)                     
    tmp_plan = list(map(lambda x : [str(tmp_prev_plan_['WW'].values[x]), str(tmp_prev_plan_['tmp_prev'].values[x])], range(0, tmp_prev_plan_.shape[0])))
    tmp_plan_join = str('/'.join(list(map(lambda x : ':'.join(tmp_plan[x]), range(0, len(tmp_plan))))))
    sale_prev = list([tmp_ww, tmp_date])#list(tmp[['WW', 'Date']].loc[row_id].values.reshape(-1))
    sale_prev.append(tmp_plan_join)
    print(i, sale_prev)
    curs.execute(insert_sale_sql, sale_prev) 
conn.commit()
conn.close()

52 134
52 [202001, '2019-12-31', '201952:60480/201951:60480/201950:60480/201949:60480/201948:60480/201947:0/201946:97331/201945:97331/201944:97331/201943:97331/201942:97331/201941:97331/201940:97331/201939:97331/201938:97331/201937:97331/201936:97331/201935:97331/201934:248531/201933:248531/201932:248531/201931:248531/201930:248531/201929:151200/201928:151200/201927:151200/201926:151200/201925:151200/201924:151200/201923:0/201922:0/201921:0/201920:0/201919:0/201918:408115/201917:408115/201916:408115/201915:408115']
53 [202002, '2020-01-07', '202001:105840/201952:105840/201951:136080/201950:136080/201949:136080/201948:136080/201947:136080/201946:264376/201945:264376/201944:264376/201943:264376/201942:264376/201941:264376/201940:264376/201939:264376/201938:264376/201937:264376/201936:264376/201935:264376/201934:264376/201933:264376/201932:264376/201931:264376/201930:264376/201929:196560/201928:196560/201927:196560/201926:196560/201925:196560/201924:196560/201923:196560/201922:196560/2019